In [1]:
from keras.layers import Input, Dense, merge
from keras.models import Model
from keras.layers import Convolution2D, MaxPooling2D, Reshape, BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

Using TensorFlow backend.


In [2]:
image_shape = (224, 224, 3)

def default_linear():
    drop = 0.1
    img_in = Input(shape=image_shape, name='img_in')
    x = img_in
    x = Convolution2D(24, (5,5), strides=(2,2), activation='relu', name="conv1")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(32, (5,5), strides=(2,2), activation='relu', name="conv2")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(64, (5,5), strides=(2,2), activation='relu', name="conv3")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu', name="conv4")(x)
    x = Dropout(drop)(x)
    x = Convolution2D(64, (3,3), strides=(1,1), activation='relu', name="conv5")(x)
    x = Dropout(drop)(x)
    
    x = Flatten(name='flattened')(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(drop)(x)
    x = Dense(50, activation='relu')(x)
    x = Dropout(drop)(x)

    outputs = []
    
    for i in range(2):
        outputs.append(Dense(1, activation='linear', name='n_outputs' + str(i))(x))
        
    model = Model(inputs=[img_in], outputs=outputs)
    return model

In [3]:
model = default_linear()
model.load_weights('models/firstPilot.h5')

Instructions for updating:
Colocations handled automatically by placer.


In [4]:
img_in = Input(shape=image_shape, name='img_in')
x = img_in
x = Convolution2D(24, (5,5), strides=(2,2), activation='relu', name='conv1')(x)
x = Convolution2D(32, (5,5), strides=(2,2), activation='relu', name='conv2')(x)
x = Convolution2D(64, (5,5), strides=(2,2), activation='relu', name='conv3')(x)
x = Convolution2D(64, (3,3), strides=(1,1), activation='relu', name='conv4')(x)
conv_5 = Convolution2D(64, (3,3), strides=(1,1), activation='relu', name='conv5')(x)
convolution_part = Model(inputs=[img_in], outputs=[conv_5])

In [5]:
for layer_num in ('1', '2', '3', '4', '5'):
    convolution_part.get_layer('conv' + layer_num).set_weights(model.get_layer('conv' + layer_num).get_weights())

In [6]:
from keras import backend as K

inp = convolution_part.input                                           # input placeholder
outputs = [layer.output for layer in convolution_part.layers[1:]]          # all layer outputs
functor = K.function([inp], outputs)

In [7]:
import tensorflow as tf

In [8]:
import numpy as np

In [9]:
import pdb

In [10]:
kernel_3x3 = tf.constant(np.array([
        [[[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]]]
]), tf.float32)

kernel_5x5 = tf.constant(np.array([
        [[[1]], [[1]], [[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]], [[1]], [[1]]], 
        [[[1]], [[1]], [[1]], [[1]], [[1]]],
        [[[1]], [[1]], [[1]], [[1]], [[1]]],
        [[[1]], [[1]], [[1]], [[1]], [[1]]]
]), tf.float32)

layers_kernels = {5: kernel_3x3, 4: kernel_3x3, 3: kernel_5x5, 2: kernel_5x5, 1: kernel_5x5}

layers_strides = {5: [1, 1, 1, 1], 4: [1, 1, 1, 1], 3: [1, 2, 2, 1], 2: [1, 2, 2, 1], 1: [1, 2, 2, 1]}

def compute_visualisation_mask(img):
#     pdb.set_trace()
    activations = functor([np.array([img])])
    activations = [np.reshape(img, (1, img.shape[0], img.shape[1], img.shape[2]))] + activations
    upscaled_activation = np.ones((21, 21))
    for layer in [5, 4, 3, 2, 1]:
        averaged_activation = np.mean(activations[layer], axis=3).squeeze(axis=0) * upscaled_activation
        output_shape = (activations[layer - 1].shape[1], activations[layer - 1].shape[2])
        x = tf.constant(
            np.reshape(averaged_activation, (1,averaged_activation.shape[0],averaged_activation.shape[1],1)),
            tf.float32
        )
        conv = tf.nn.conv2d_transpose(
            x, layers_kernels[layer],
            output_shape=(1,output_shape[0],output_shape[1], 1), 
            strides=layers_strides[layer], 
            padding='VALID'
        )
        with tf.Session() as session:
            result = session.run(conv)
        upscaled_activation = np.reshape(result, output_shape)
    final_visualisation_mask = upscaled_activation
    return (final_visualisation_mask - np.min(final_visualisation_mask))/(np.max(final_visualisation_mask) - np.min(final_visualisation_mask))

In [11]:
import numpy as np

In [12]:
from matplotlib import pyplot as plt
%matplotlib inline

In [13]:
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import display, HTML

def plot_movie_mp4(image_array):
    dpi = 72.0
    xpixels, ypixels = image_array[0].shape[0], image_array[0].shape[1]
    fig = plt.figure(figsize=(ypixels/dpi, xpixels/dpi), dpi=dpi)
    im = plt.figimage(image_array[0])

    def animate(i):
        im.set_array(image_array[i])
        return (im,)

    anim = animation.FuncAnimation(fig, animate, frames=len(image_array))
    anim.save("viz.mp4")
    display(HTML(anim.to_html5_video()))

In [14]:
from glob import iglob

In [15]:
imgs = []
alpha = 0.004
beta = 1.0 - alpha
counter = 0
for path in sorted(iglob('data/tub_11_20-03-29/*.jpg')):
    img = plt.imread(path)
    salient_mask = compute_visualisation_mask(img)
    salient_mask_stacked = np.dstack((salient_mask,salient_mask))
    salient_mask_stacked = np.dstack((salient_mask_stacked,salient_mask))
    blend = img.astype('float32') * alpha + salient_mask_stacked * beta
    imgs.append(blend)
    counter += 1
    if counter >= 400:
        break

InvalidArgumentError: Conv2DSlowBackpropInput: Size of out_backprop doesn't match computed: actual = 23, computed = 12 spatial_dim: 1 input: 25 filter: 3 output: 23 stride: 2 dilation: 1
	 [[node conv2d_transpose_1 (defined at <ipython-input-10-94a9fc6d8b30>:35) ]]
	 [[node conv2d_transpose_1 (defined at <ipython-input-10-94a9fc6d8b30>:35) ]]

Caused by op 'conv2d_transpose_1', defined at:
  File "/home/parker/.conda/envs/donkey/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/parker/.conda/envs/donkey/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/parker/.conda/envs/donkey/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/parker/.conda/envs/donkey/lib/python3.7/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/home/parker/.conda/envs/donkey/lib/python3.7/site-packages/ipykernel/

In [16]:
plot_movie_mp4(imgs)

IndexError: list index out of range